Augmentation performed based on https://arxiv.org/pdf/1812.10595.pdf

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("/storage"))

# Any results you write to the current directory are saved as output.

['retinopathy-train-2015', 'lost+found', 'aptosplus', 'diabetic-retinopathy-resized', 'aptos2019-blindness-detection', 'resnet101.pth']


In [2]:
import os, sys
import re
from multiprocessing import Pool
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import skimage.io
from skimage.transform import resize
from scipy import ndimage
from imgaug import augmenters as iaa
from tqdm import tqdm
import PIL
from PIL import Image, ImageOps
import cv2
from sklearn.utils import class_weight, shuffle
from sklearn.metrics import f1_score, fbeta_score
from sklearn.model_selection import train_test_split
import seaborn as sns
from collections import Counter
import imageio

from torchvision.transforms import transforms

WORKERS = 2
CHANNEL = 3

import warnings
warnings.filterwarnings("ignore")
IMG_SIZE = 512
NUM_CLASSES = 5
SEED = 77
TRAIN_NUM = 1000 # use 1000 when you just want to explore new idea, use -1 for full train

In [3]:
df_train = pd.read_csv('/storage/aptosplus/trainLabels.csv')

In [4]:
df_train.groupby('diagnosis').agg(['count'])

,id_code,ds
,count,count
diagnosis,,
0,27615,27615
1,2813,2813
2,6291,6291
3,1066,1066
4,1003,1003


In [5]:
df_train.tail()

,id_code,diagnosis,ds
38783,44347_right,0,eyepacs
38784,44348_left,0,eyepacs
38785,44348_right,0,eyepacs
38786,44349_left,0,eyepacs
38787,44349_right,1,eyepacs


1 augmentation == 1 extra image

if d == 0 0 augmentations   

if d == 1 10 transformations  

if d == 2 4 transformations    

if d == 3 25 transformations  

if d == 4 25 transformations  


In [6]:
n_augmentation_dict = {
    0:0,
    1:10,
    2:4,
    3:25,
    4:25
}

### csv

In [9]:
def add_aug_rows(packed_args):
    i, row = packed_args

    augs = []
    
    for nth_aug in range(n_augmentation_dict[row['diagnosis']]):
        augs += [{'id_code': f'{row["id_code"]}_a{nth_aug}', 'diagnosis': row["diagnosis"], 'ds': row["ds"]}]
    
    return augs

In [10]:
%%time

pool = Pool()                   

augmentations = pool.map(add_aug_rows, df_train.iterrows())

CPU times: user 7.31 s, sys: 164 ms, total: 7.47 s
Wall time: 7.6 s


In [13]:
augmentations = [item for sublist in augmentations for item in sublist]

In [18]:
df_train_aug = df_train.copy()

In [19]:
df_train_aug = df_train_aug.append(augmentations)

In [20]:
df_train_aug.tail()

,id_code,diagnosis,ds
105014,44349_right_a5,1,eyepacs
105015,44349_right_a6,1,eyepacs
105016,44349_right_a7,1,eyepacs
105017,44349_right_a8,1,eyepacs
105018,44349_right_a9,1,eyepacs


In [23]:
!ls /storage/aptosplus

aptos  eyepacs	trainLabels.csv


In [24]:
df_train_aug.to_csv('/storage/aptosplus/trainLabels_aug.csv', index=False)

### images

In [7]:
jitter=(0.6, 1.4)

transform = transforms.Compose([
        transforms.RandomResizedCrop(448, scale=(0.85, 0.95)),
        transforms.ColorJitter(brightness=jitter, contrast=jitter, saturation=jitter, hue=.1),
        transforms.RandomHorizontalFlip(p=0.5),
        transforms.RandomVerticalFlip(p=0.5),
        transforms.RandomAffine((0, 360), shear=(20, 200), scale=(1/1.3, 1.3), translate=(25/512, 25/512))
#         transforms.Normalize(mean=(123.68, 116.779, 103.939), std=(58.393, 57.12, 57.375))
    ])

def add_aug_images(packed_args):
    i, row = packed_args
    
    for nth_aug in range(n_augmentation_dict[row['diagnosis']]):
        image = None
        
        if row['ds'] == 'aptos':
            with Image.open(f'/storage/aptosplus/aptos/{row["id_code"]}.png') as img:
                image = transform(img)
                image.save(f'/storage/aptosplus/aptos/{row["id_code"]}_a{nth_aug}.png')
        else:
            with Image.open(f'/storage/aptosplus/eyepacs/{row["id_code"]}.png') as img:
                image = transform(img)
                image.save(f'/storage/aptosplus/eyepacs/{row["id_code"]}_a{nth_aug}.png')   

In [ ]:
%%time

pool = Pool()                   
pool.map(add_aug_images, df_train.iterrows())